In [1]:
import pandas as pd
import numpy as np
import pymysql

from datetime import datetime, timedelta

def excel_a_fecha(valor_excel):
    # Convertir a fecha
    fecha_excel = datetime(1899, 12, 30) + timedelta(days=valor_excel)
    
    # Devolver la fecha
    return fecha_excel

In [5]:
cnx = pymysql.connect(host='159.89.90.197', port=3306, user='antonio_diaz', passwd="4nT0ni_0.dZ",
                           charset='utf8',db = 'volana')

In [58]:
sql = """
select * from
cartera_distribuidor
where fecha = '2023-11-30'
"""
cursor = cnx.cursor()
cursor.execute(sql)
data_cartera = cursor.fetchall()
data_cartera = pd.DataFrame(data_cartera, columns=[i[0] for i in cursor.description])

In [59]:
data_cartera['id_distribuidor'].nunique()

16836

In [60]:
data_riesgo_ant = pd.read_csv("./data/riesgo_cartera_10_31_23.csv")

In [61]:
data_riesgo_ant['ID DIST.'].nunique()

15761

In [62]:
faltantes = np.setdiff1d(data_cartera['id_distribuidor'].unique(), data_riesgo_ant['ID DIST.'].unique())

In [63]:
data_riesgo_ant

,ID DIST.,probs,grupo
0,1,80.19,2
1,19,8.85,10
2,29,7.95,10
3,38,17.77,9
4,51,98.70,1
...,...,...,...
15756,166935,53.96,5
15757,166945,26.13,8
15758,166991,51.45,5
15759,167004,79.69,2


In [64]:
data_cartera['capital'] = data_cartera.apply(lambda row: row['total_pagado_capital'] + row['saldo_pendiente_capital'],
                                             axis = 1)

In [65]:
data_cosecha = data_cartera.groupby('id_distribuidor').agg({'capital':'sum', 'saldo_pendiente_capital':'sum'}).reset_index()
data_cosecha = data_cosecha.loc[[k in faltantes for k in data_cosecha['id_distribuidor']]]

In [66]:
data_cosecha['perdida_cosecha'] = 1 - (data_cosecha['capital'] - data_cosecha['saldo_pendiente_capital']) /  data_cosecha['capital']

In [67]:
ids_sin_suficiente_info = data_cartera.loc[data_cartera.fecha_aut_linea > '2023-11-01'].id_distribuidor

In [68]:
data_cosecha.perdida_cosecha.mean()

0.5687556595999544

In [69]:
data_cosecha = data_cosecha[['id_distribuidor', 'perdida_cosecha']]

In [70]:
#data_cartera.loc[data_cartera['ID DIST.'] ==  40991]

In [71]:
def setcut(cosecha_):
    #p = p*100
    cosecha_ *= 100

    grupo = 1
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00]))
    cosechas = np.array(sorted([0, 1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69], reverse = True))

    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))

    enter_ = False
    for cosecha in cosechas:
        if cosecha <= cosecha_:

            enter_ = True
            break

    if enter_:
        grupo = np.where(cosechas == cosecha)[0][0] 

    return grupo, cuts[grupo], cosechas[grupo]

In [72]:
def setcut_prob(p, grupo_ = True):
    p = p*100
    cuts = np.array(sorted([100, 81.09, 72.06, 63.19, 54.27, 45.11, 36.10, 27.47, 18.17, 9.05, 0.00]))
    cosechas = np.array(sorted([1.54, 3.4, 4.97, 7.38, 10.69, 11.87, 12.35, 12.68, 20.33, 22.69], reverse = True))
    
    #cuts = np.array(sorted([1, 0.9919, 0.975, 0.953, 0.9196, 0.8717, 0.7832, 0.6737, 0.5072, .2738]))*100
    #cosechas = np.array(sorted([2.31, 2.63, 3.1, 3.12, 3.94, 4.2, 4.33, 5.15, 5.71, 59.82], reverse = True))
    
    for cut in cuts:
        if cut > p:
            break
            
    grupo = np.where(cuts == cut)[0][0] 
    if grupo_:
        return grupo
    else:
        return cosechas[grupo - 1]

In [73]:
setcut(1.6)

(0, 0.0, 22.69)

In [74]:
_, data_cosecha['probs'], data_cosecha['cosecha_canasta'] = zip(*data_cosecha.perdida_cosecha.apply(setcut))

In [75]:
data_cartera.query("id_distribuidor == 171193")#[['capital', 'total_pagado_capital', 'saldo_pendiente_capital']]

,sucursal,id_coordinador,coordinador,id_asesor,asesor,id_distribuidor,distribuidor,fecha_aut_linea,fecha_ult_incremento,no_incrementos,...,moratorio_vencido,seguro_distribuidor_vencido,seguro_asociado_vencido,linea_bloqueada,id_cliente,tipo_distribuidor_origen,fecha_cambio_tipo_distribuidor,fecha_consulta_cc_1,fecha_consulta_cc_2,capital
16831,CHIMALHUACAN 2,33544,FERNANDO ORTIZ CORONEL,30967,LIZBETH GUADALUPE MUJICA HERNANDEZ,171193,LAS PRINCESAS 2 - REESTRUCTURA,2023-11-30,NaT,0,...,NaN,NaN,NaN,None,None,None,NaT,NaT,NaT,44721.73


In [83]:
data_cosecha.loc[[k in ids_sin_suficiente_info for k in data_cosecha.id_distribuidor]]

,id_distribuidor,probs,cosecha_canasta,grupo
677,13077,0.9130,1.54,10
683,13186,0.6357,4.97,8
846,16665,0.9146,1.54,10
855,16824,0.9400,1.54,10


In [77]:
def move_prob(val):
    val /= 100
    if val == 0:
        val += np.random.randint(0, 200)/1000
    elif val == 1:
        val -= np.random.randint(0, 200)/1000
    else:
        val += np.random.randint(-200, 200)/1000
    return val

In [78]:
data_cosecha.probs = data_cosecha.probs.apply(move_prob)

In [79]:
data_cosecha['grupo'] = data_cosecha.probs.apply(setcut_prob, grupo_ = True)

In [80]:
data_cosecha['cosecha_canasta'] = data_cosecha.probs.apply(setcut_prob, grupo_ = False)

In [81]:
data_cosecha.drop("perdida_cosecha", axis = 1, inplace=True)

In [82]:
data_cosecha

,id_distribuidor,probs,cosecha_canasta,grupo
489,9006,0.1930,12.68,3
539,10033,0.6009,7.38,7
540,10036,0.8036,3.40,9
542,10091,0.1677,20.33,2
558,10399,0.3751,11.87,5
...,...,...,...,...
16831,171193,0.0240,22.69,1
16832,171201,0.1570,20.33,2
16833,171203,0.0820,22.69,1
16834,171205,0.1510,20.33,2


In [87]:
data_riesgo_ant.probs = data_riesgo_ant.probs / 100

In [88]:
data_riesgo_ant['cosecha_canasta'] = data_riesgo_ant.probs.apply(setcut_prob, grupo_ = False)

In [96]:
data_riesgo_ant.rename(columns={'ID DIST.': 'id_distribuidor'}, inplace = True)
data_riesgo_ant = data_riesgo_ant[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']];

In [97]:
data_cosecha = data_cosecha[['id_distribuidor', 'probs', 'grupo', 'cosecha_canasta']]

In [98]:
data_final = pd.concat([data_riesgo_ant, data_cosecha])

In [99]:
data_final

,id_distribuidor,probs,grupo,cosecha_canasta
0,1,0.8019,2,3.40
1,19,0.0885,10,22.69
2,29,0.0795,10,22.69
3,38,0.1777,9,20.33
4,51,0.9870,1,1.54
...,...,...,...,...
16831,171193,0.0240,1,22.69
16832,171201,0.1570,2,20.33
16833,171203,0.0820,1,22.69
16834,171205,0.1510,2,20.33


In [100]:
data_final.to_csv("./data/riesgo_cartera_11_30_23.csv")

In [102]:
data_final.query('id_distribuidor==171193')

,id_distribuidor,probs,grupo,cosecha_canasta
16831,171193,0.024,1,22.69


In [101]:
data_final.corr()

,id_distribuidor,probs,grupo,cosecha_canasta
id_distribuidor,1.000000,-0.174859,0.001428,0.104476
probs,-0.174859,1.000000,-0.743561,-0.913958
grupo,0.001428,-0.743561,1.000000,0.724146
cosecha_canasta,0.104476,-0.913958,0.724146,1.000000
